## Decision Tree example
Let's explore implementation of Decision Tree!

In [1]:
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """Given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2)
               for p in class_probabilities
               if p > 0)                     # ignore zero probabilities

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])



### Entropy of a partition
One problem with this approach is that partitioning by an attribute with many different values will result in a very low entropy due to overfitting.

In [2]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """Returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum(data_entropy(subset) * len(subset) / total_count
               for subset in subsets)

### Creating a decision tree
The interviewee data, consisting of (per your specification) a NamedTuple of the relevant attributes for each candidate—her level, her preferred language, whether she is active on Twitter, whether she has a PhD, and whether she interviewed well.

In [4]:
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  # allow unlabeled data
    class_label: Optional[str] = None  # allow unlabeled data

                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

In [5]:
from typing import Dict, TypeVar
from collections import defaultdict

T = TypeVar('T')  # generic type for inputs

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists based on the specified attribute."""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)  # value of the specified attribute
        partitions[key].append(input)    # add input to the correct partition
    return partitions

print(partition_by(inputs, 'level'))

def partition_entropy_by(inputs: List[Any],
                         attribute: str,
                         label_attribute: str) -> float:
    """Compute the entropy corresponding to the given partition"""
    # partitions consist of our inputs
    partitions = partition_by(inputs, attribute)

    # but partition_entropy needs just the class labels
    labels = [[getattr(input, label_attribute) for input in partition]
              for partition in partitions.values()]
    #print(labels)

    return partition_entropy(labels)

for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well'))

assert 0.69 < partition_entropy_by(inputs, 'level', 'did_well')  < 0.70
assert 0.86 < partition_entropy_by(inputs, 'lang', 'did_well')   < 0.87
assert 0.78 < partition_entropy_by(inputs, 'tweets', 'did_well') < 0.79
assert 0.89 < partition_entropy_by(inputs, 'phd', 'did_well')    < 0.90



defaultdict(<class 'list'>, {'Senior': [Candidate(level='Senior', lang='Java', tweets=False, phd=False, did_well=False, class_label=None), Candidate(level='Senior', lang='Java', tweets=False, phd=True, did_well=False, class_label=None), Candidate(level='Senior', lang='Python', tweets=False, phd=False, did_well=False, class_label=None), Candidate(level='Senior', lang='R', tweets=True, phd=False, did_well=True, class_label=None), Candidate(level='Senior', lang='Python', tweets=True, phd=True, did_well=True, class_label=None)], 'Mid': [Candidate(level='Mid', lang='Python', tweets=False, phd=False, did_well=True, class_label=None), Candidate(level='Mid', lang='R', tweets=True, phd=True, did_well=True, class_label=None), Candidate(level='Mid', lang='Python', tweets=False, phd=True, did_well=True, class_label=None), Candidate(level='Mid', lang='Java', tweets=True, phd=False, did_well=True, class_label=None)], 'Junior': [Candidate(level='Junior', lang='Python', tweets=False, phd=False, did_we

In [6]:
# the lowest is level in previous step. Now we’ll need to make a subtree for each possible level value
senior_inputs = [input for input in inputs if input.level == 'Senior']

assert 0.4 == partition_entropy_by(senior_inputs, 'lang', 'did_well')
assert 0.0 == partition_entropy_by(senior_inputs, 'tweets', 'did_well')
assert 0.95 < partition_entropy_by(senior_inputs, 'phd', 'did_well') < 0.96

# so which should we pick now? 
# Yes, it is tweets! So you will continue to split the tree

In [7]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://learning.oreilly.com/library/view/data-science-from/9781492041122/assets/dsf2_1703.png")
# this is the decision tree finally you would get

### Putting It All Together
Now that we’ve seen how the algorithm works, we would like to implement it more generally. 
<br>We define a tree to be either:
<br>a Leaf (that predicts a single value), or
<br>a Split (containing an attribute to split on, subtrees for specific values of that attribute, and possibly a default value to use if we see an unknown value).

In [8]:
from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]


In [9]:
# With this representation, our hiring tree would look like:

hiring_tree = Split('level', {   # First, consider "level".
    'Junior': Split('phd', {     # if level is "Junior", next look at "phd"
        False: Leaf(True),       #   if "phd" is False, predict True
        True: Leaf(False)        #   if "phd" is True, predict False
    }),
    'Mid': Leaf(True),           # if level is "Mid", just predict True
    'Senior': Split('tweets', {  # if level is "Senior", look at "tweets"
        False: Leaf(False),      #   if "tweets" is False, predict False
        True: Leaf(True)         #   if "tweets" is True, predict True
    })
})

#There’s still the question of what to do if we encounter an unexpected (or missing) attribute value. 
#What should our hiring tree do if it encounters a candidate whose level is Intern? 
#We’ll handle this case by populating the default_value attribute with the most common label.

### Classify and build tree



In [10]:
def classify(tree: DecisionTree, input: Any) -> Any:
    """classify the input using the given decision tree"""

    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value

    # Otherwise this tree consists of an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values of are subtrees to consider next
    subtree_key = getattr(input, tree.attribute)

    if subtree_key not in tree.subtrees:   # If no subtree for key,
        return tree.default_value          # return the default value.

    subtree = tree.subtrees[subtree_key]   # Choose the appropriate subtree
    return classify(subtree, input)        # and use it to classify the input.

In [11]:
def build_tree_id3(inputs: List[Any],
                   split_attributes: List[str],
                   target_attribute: str) -> DecisionTree:
    # Count target labels
    label_counts = Counter(getattr(input, target_attribute)
                           for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]

    # If there's a unique label, predict it
    if len(label_counts) == 1:
        return Leaf(most_common_label)

    # If no split attributes left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)

    # Otherwise split by the best attribute

    def split_entropy(attribute: str) -> float:
        """Helper function for finding the best attribute"""
        return partition_entropy_by(inputs, attribute, target_attribute)

    best_attribute = min(split_attributes, key=split_entropy)

    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]

    # recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset,
                                                 new_attributes,
                                                 target_attribute)
                for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)

tree = build_tree_id3(inputs,
                      ['level', 'lang', 'tweets', 'phd'],
                      'did_well')

# Should predict True
assert classify(tree, Candidate("Junior", "Java", True, False))


#flowerTree("short", "medium", "short","short")

# Should predict False
assert not classify(tree, Candidate("Junior", "Java", True, True))

# data with unexpected values
# Should predict True
assert classify(tree, Candidate("Intern", "Java", True, True))

### Home work practice in pair
1. Please explore and understand the decision tree theory and code<br>
2. (8 points)We explored the iris dataset (you could load it from sklearn) using several machine learning model, now could you please use the code we learned today about decision tree to train and make predictions for the following: 
<br>['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
<br>[6.0, 3.0, 5.0, 0.6]
<br>[6.0, 3.0, 5.0, 1.6]
<br>[6.0, 3.0, 5.0, 2.6]
<br>Hint: the attributes in iris dataset is floating number, but you could always convert that into classes by setting bins. For example, petal width, you could have three different classes: [0,0.8], (0.8, 1.75], (1.75, infinite). 
3. (2 points) Sklearn has great tutorial about decision tree (https://scikit-learn.org/stable/modules/tree.html), could you please train a model on iris dataset and make predictions as previous question? 


In [13]:
# let's first download it
import requests

data = requests.get(
  "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
)

with open('iris.dat', 'w') as f:
    f.write(data.text)

# The data is comma-separated, with fields: sepal_length, sepal_width, petal_length, petal_width, class
# For example, the first line looks: 5.1,3.5,1.4,0.2,Iris-setosa

In [14]:
class Flower(NamedTuple):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
    class_label: Optional[str] = None  # allow unlabeled data

In [15]:
content = []
with open('iris.dat', 'r') as f:
    for line in f:
        data = line.strip().split(',')
        if data[0] == '':
            break

        sepal_length = 1
        if float(data[0]) > 6.7:
            sepal_length = 3
        elif float(data[0]) < 5.5:
            sepal_length = 2

        sepal_width = 1
        if float(data[0]) > 3.6:
            sepal_width = 3
        elif float(data[0]) < 2.8:
            sepal_width = 2

        petal_length = 1
        if float(data[0]) > 5.0:
            petal_length = 3
        elif float(data[0]) < 3.0:
            petal_length = 2

        petal_width = 1
        if float(data[0]) > 1.7:
            petal_width = 3
        elif float(data[0]) < 0.9:
            petal_width = 2
        
        flower = Flower(
            sepal_length,
            sepal_width,
            petal_length,
            petal_width,
            class_label=data[4]
        )
        content.append(flower)

In [16]:
content

[Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length=2, sepal_width=3, petal_length=1, petal_width=3, class_label='Iris-setosa'),
 Flower(sepal_length

In [20]:
sepal_lengths = [flower.sepal_length for flower in content]
print(max(sepal_lengths))
print(min(sepal_lengths))

sepal_widths = [flower.sepal_width for flower in content]
print(max(sepal_widths))
print(min(sepal_widths))

petal_lengths = [flower.petal_length for flower in content]
print(max(petal_lengths))
print(min(petal_lengths))

petal_widths = [flower.petal_width for flower in content]
print(max(petal_widths))
print(min(petal_widths))


3
1
3
3
1
1
3
3


In [23]:
tree = build_tree_id3(content,
                      ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
                      'class_label')

In [29]:
print(classify(tree, Flower(5.1, 3.5, 1.4, 0.2)))

Iris-setosa


In [25]:
assert classify(tree, Flower(5.1, 3.5, 1.4, 0.2)) == 'Iris-setosa'

In [39]:
#sklearn decision tree
from sklearn import tree

X = []
Y = []
for flower in content:
    sepal_length = flower.sepal_length
    sepal_width = flower.sepal_width
    petal_length = flower.petal_length
    petal_width = flower.petal_width
    
    class_label = flower.class_label
    #will use 1 as Iris-setosa
    class_label_to_integer = 1
    if class_label_to_integer == "Iris-versicolor":
        class_label_to_integer = 2
    elif class_label_to_integer == "Iris-virginica":
        class_label_to_integer = 3
    
    X.append([sepal_length, sepal_width, petal_length, petal_width])
    Y.append(class_label_to_integer)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

integer_output = clf.predict([[2.0, 2.0, 2.0, 2.0]])
if integer_output == 1:
    flower_output = "Iris_setosa"
elif integer_output == 2:
    flower_output = "Iris-versicolor"
else:
    flower_output = "Iris-virginica"
    
print(flower_output)

Iris_setosa
